<a href="https://colab.research.google.com/github/AradhanaMahati/Musical-Source-Separation/blob/main/ICA_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
np.random.seed(0)

from scipy.io import wavfile as wf
import librosa

In [10]:
#guitar-mix2.wav and piano-mix1.wav : both are in D with 100bpm
#sound1 size < sound2 size

sound1, sr1 = librosa.load("/content/drive/MyDrive/audio/drum-mix1.wav", mono=False)
print(sound1.shape)
sound1 = librosa.to_mono(sound1)
print(sound1.shape)

sound2, sr2 = librosa.load("/content/drive/MyDrive/audio/guitar-mix1.wav", mono=False)
print(sound2.shape)
sound2 = librosa.to_mono(sound2)
print(sound2.shape)


(2, 336856)
(336856,)
(2, 423360)
(423360,)


In [11]:
def mix_sources(sources, apply_noise=False):
  for i in range(len(sources)):
      max_val = np.max(sources[i])
      if(max_val > 1 or np.min(sources[i]) < 1):
          sources[i] = sources[i] / (max_val / 2) - 0.5

  mixture = np.c_[[source for source in sources]]

  if(apply_noise):
      mixture += 0.02 * np.random.normal(size=mixture.shape)

  return mixture


x = mix_sources([sound1, sound2[:sound1.shape[0]]], False)
print(f'Shape of sound1: {sound1.shape}, sound2: {sound1.shape}, Linear Mix: {x.shape}')
#wf.write('/content/drive/MyDrive/audio/drum&guitar_mix1.wav', sr2, x.mean(axis=0).astype(np.float32))

Shape of sound1: (336856,), sound2: (336856,), Linear Mix: (2, 336856)


In [12]:
def center(x):
    x = np.array(x)
    return x - x.mean(axis=1, keepdims=True)

def whiten(x):
  eigen_values, eigen_vectors = np.linalg.eigh(np.cov(x))
  D = np.diag(eigen_values)
  sqrt_inverse_D = np.sqrt(np.linalg.inv(D))
  x_whiten = eigen_vectors @ (sqrt_inverse_D @ (eigen_vectors.T @ x))

  print(f'Shape of Eigen Values: {eigen_values.shape}, Eigen Vectors: {eigen_vectors.shape}, Whitened Data: {x_whiten.shape}')

  return x_whiten, D, eigen_vectors


X_whiten, D, E = whiten(center(x))
print("D: ",D)
print()
print("E: ",E)

Shape of Eigen Values: (2,), Eigen Vectors: (2, 2), Whitened Data: (2, 336856)
D:  [[0.10990851 0.        ]
 [0.         0.34631188]]

E:  [[-0.00487041 -0.99998814]
 [-0.99998814  0.00487041]]


In [13]:
def objFunc(x):
    return np.tanh(x)

def dObjFunc(x):
    return 1 - (objFunc(x) ** 2)

def calc_w_hat(W, X):
    # Implementation of the eqn. Towards Convergence
    w_hat = (X * objFunc(W.T @ X)).mean(axis=1) - dObjFunc(W.T @ X).mean() * W
    w_hat /= np.sqrt((w_hat ** 2).sum())

    return w_hat

In [14]:
def ica(X, iterations, tolerance=1e-5):
    num_components = X.shape[0]

    W = np.zeros((num_components, num_components), dtype=X.dtype)
    distances = {i: [] for i in range(num_components)}

    for i in np.arange(num_components):
        w = np.random.rand(num_components)
        for j in np.arange(iterations):
            w_new = calc_w_hat(w, X)
            if(i >= 1):
                w_new -= np.dot(np.dot(w_new, W[:i].T), W[:i])
            distance = np.abs(np.abs((w * w_new).sum()) - 1)

            w = w_new
            if(distance < tolerance):
                print(f'Convergence attained for the {i+1}/{num_components} component.')
                print(f'Component: {i+1}/{num_components}, Step: {j}/{iterations}, Distance: {distance}\n')

                break;

            distances[i].append(distance)

            if(j % 50 == 0):
                print(f'Component: {i+1}/{num_components}, Step: {j}/{iterations}, Distance: {distance}')



        W[i, :] = w
    S = np.dot(W, X)

    return S, distances

S, distances = ica(X_whiten, iterations=100)
wf.write('/content/drive/MyDrive/audio/guitar-mix1_predicted.wav', sr1, S[0].astype(np.float32))
wf.write('/content/drive/MyDrive/audio/drum-mix1_predicted.wav', sr2, S[1].astype(np.float32))

Component: 1/2, Step: 0/100, Distance: 0.21744583017916386
Convergence attained for the 1/2 component.
Component: 1/2, Step: 3/100, Distance: 4.65189372578223e-08

Component: 2/2, Step: 0/100, Distance: 0.8167487256570896
Component: 2/2, Step: 50/100, Distance: 0.0002113583254977902
